**Capacited Vehicle Routing Problem**

**Optimizing VRP Using Numerical Methods -**
**1. Mixed Integer Programming**
**2. Nearest Neighbor**
**3. Two-opt Algorithm**

Dataset - Google OR Tools - https://developers.google.com/optimization/routing/cvrp


In [ ]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    #Location Matrix : 17 including Depot
    data["distance_matrix"] = [
        # fmt: off
      [0, 548, 776, 696, 582, 274, 502, 194, 308, 194, 536, 502, 388, 354, 468, 776, 662],
      [548, 0, 684, 308, 194, 502, 730, 354, 696, 742, 1084, 594, 480, 674, 1016, 868, 1210],
      [776, 684, 0, 992, 878, 502, 274, 810, 468, 742, 400, 1278, 1164, 1130, 788, 1552, 754],
      [696, 308, 992, 0, 114, 650, 878, 502, 844, 890, 1232, 514, 628, 822, 1164, 560, 1358],
      [582, 194, 878, 114, 0, 536, 764, 388, 730, 776, 1118, 400, 514, 708, 1050, 674, 1244],
      [274, 502, 502, 650, 536, 0, 228, 308, 194, 240, 582, 776, 662, 628, 514, 1050, 708],
      [502, 730, 274, 878, 764, 228, 0, 536, 194, 468, 354, 1004, 890, 856, 514, 1278, 480],
      [194, 354, 810, 502, 388, 308, 536, 0, 342, 388, 730, 468, 354, 320, 662, 742, 856],
      [308, 696, 468, 844, 730, 194, 194, 342, 0, 274, 388, 810, 696, 662, 320, 1084, 514],
      [194, 742, 742, 890, 776, 240, 468, 388, 274, 0, 342, 536, 422, 388, 274, 810, 468],
      [536, 1084, 400, 1232, 1118, 582, 354, 730, 388, 342, 0, 878, 764, 730, 388, 1152, 354],
      [502, 594, 1278, 514, 400, 776, 1004, 468, 810, 536, 878, 0, 114, 308, 650, 274, 844],
      [388, 480, 1164, 628, 514, 662, 890, 354, 696, 422, 764, 114, 0, 194, 536, 388, 730],
      [354, 674, 1130, 822, 708, 628, 856, 320, 662, 388, 730, 308, 194, 0, 342, 422, 536],
      [468, 1016, 788, 1164, 1050, 514, 514, 662, 320, 274, 388, 650, 536, 342, 0, 764, 194],
      [776, 868, 1552, 560, 674, 1050, 1278, 742, 1084, 810, 1152, 274, 388, 422, 764, 0, 798],
      [662, 1210, 754, 1358, 1244, 708, 480, 856, 514, 468, 354, 844, 730, 536, 194, 798, 0],
        # fmt: on
    ]
    data["num_vehicles"] = 4
    #Depot = central location
    data["depot"] = 0
    #Demand Vector : Demand for every location. The demand of depot (0) is 0.
    data["demands"] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
    data["vehicle_capacities"] = [15, 15, 15, 15]
    return data

In [3]:
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, value

def solve_cvrp_with_mip(data):
    # Extract data
    distance_matrix = data["distance_matrix"]
    demands = data["demands"]
    vehicle_capacities = data["vehicle_capacities"]
    num_vehicles = data["num_vehicles"]
    depot = data["depot"]

    # Number of locations
    n = len(distance_matrix)

    # Number of vehicles
    m = num_vehicles

    # Capacity of each vehicle
    Q = vehicle_capacities[0]

    # Create a binary variable to indicate if vehicle i travels from location j to location k
    x = LpVariable.dicts("x", ((i, j, k) for i in range(1, m + 1) for j in range(n) for k in range(n)), cat='Binary')

    # Create a continuous variable representing the cumulative load on vehicle i after visiting location j
    u = LpVariable.dicts("u", (i for i in range(1, m + 1)), lowBound=0, upBound=Q, cat='Continuous')

    # Create the CVRP problem
    cvrp_problem = LpProblem("CVRP", LpMinimize)

    # Objective function
    cvrp_problem += lpSum(distance_matrix[j][k] * x[i, j, k] for i in range(1, m + 1) for j in range(n) for k in range(n)), "Total Distance"

    # Constraints
    # Each location must be visited exactly once
    for j in range(1, n - 1):
        cvrp_problem += lpSum(x[i, j, k] for i in range(1, m + 1) for k in range(1, n)) == 1, f"Visit_once_in{j}"

    for k in range(1, n - 1):
        cvrp_problem += lpSum(x[i, j, k] for i in range(1, m + 1) for j in range(1, n)) == 1, f"Visit_once_out{k}"

    # Each vehicle must leave and return to the depot
    for i in range(1, m + 1):
        cvrp_problem += lpSum(x[i, j, 0] for j in range(1, n)) == 1, f"Leave_depot_{i}"
        cvrp_problem += lpSum(x[i, 0, k] for k in range(1, n)) == 1, f"Return_depot_{i}"

    # Subtour elimination to avoid cycles
    for i in range(1, m + 1):
        for j in range(1, m+1):
            for k in range(1, n):
                if j != k:
                    cvrp_problem += u[i] - u[j] + Q * x[i, j, k] <= Q - demands[k], f"Subtour_elim_{i}_{j}_{k}"


    # Capacity constraint for each vehicle
    for i in range(1, m + 1):
        for j in range(1, n):
            cvrp_problem += u[i] + demands[j] <= Q, f"Capacity_{i}_{j}"

    # Solve the problem
    cvrp_problem.solve()

    # Print the status
    #print("Status:", LpProblem.status[cvrp_problem.status])

    if cvrp_problem.status == 1:  # 1 corresponds to LpStatusOptimal
        print("Optimal Solution:")
        for i in range(1, m + 1):
            route = [0]  # Start at the depot
            for j in range(n):
                for k in range(n):
                    if value(x[i, j, k]) == 1:
                        route.append(k)
            route.append(0)  # Return to the depot
            print(f"Vehicle {i}: {route}")

In [ ]:
def nearest_neighbor_algorithm(data):

    distance_matrix = data["distance_matrix"]
    num_vehicles = data["num_vehicles"]
    depot = data["depot"]
    customer_demands = data["demands"]
    vehicle_capacities = data["vehicle_capacities"]

    #Initializing routes, total distances, and total distance across all routes
    routes = []
    total_distances = []
    total_distance_all_routes = 0

    #Creating a copy of demands to keep track of remaining demand for each location
    remaining_demands = customer_demands.copy()

    for vehicle in range(num_vehicles):
        current_capacity = vehicle_capacities[vehicle]
        current_route = [depot]
        current_location = depot
        total_distance = 0

        #Until the vehicle's capacity is exhausted find the nearest neighbor with remaining demand
        while current_capacity > 0:
            candidates = [ index for index, demand in enumerate(remaining_demands) if demand > 0 and current_capacity >= customer_demands[index]]
            if not candidates:
                break

            nearest_neighbor = min(candidates,key=lambda x: distance_matrix[current_location][x])

            #Updating the current route, capacity, and distance
            current_route.append(nearest_neighbor)
            current_capacity -= customer_demands[nearest_neighbor]
            total_distance += distance_matrix[current_location][nearest_neighbor]
            current_location = nearest_neighbor

            #Updating the demand of the selected customer as satisfied
            remaining_demands[nearest_neighbor] = 0

        total_distance += distance_matrix[current_location][depot]
        #Return to the depot when all the nodes in the route are visited
        current_route.append(depot)

        routes.append(current_route)
        total_distances.append(total_distance)

        total_distance_all_routes += total_distance

    return routes, total_distances, total_distance_all_routes

if __name__ == "__main__":

    data = create_data_model()
    routes, total_distances, total_distance_all_routes = nearest_neighbor_algorithm(data)

    print("Nearest Neighbor Solution:")

    for i, (route, distance) in enumerate(zip(routes, total_distances)):
        print(f"Route {i + 1}: {route}, Total Distance: {distance}")

    print(f"\nTotal Distance Across All Routes: {total_distance_all_routes}")


Nearest Neighbor Solution:
Route 1: [0, 7, 5, 6, 2, 0], Total Distance: 1780
Route 2: [0, 9, 8, 14, 10, 0], Total Distance: 1712
Route 3: [0, 13, 12, 11, 15, 0], Total Distance: 1712
Route 4: [0, 1, 4, 3, 16, 0], Total Distance: 2876

Total Distance Across All Routes: 8080


In [ ]:
def two_opt_algorithm(route, distance_matrix):
    best_route = route

    improved = True

    while improved:
        improved = False
        for i in range(1, len(best_route) - 1):
            for j in range(i + 1, len(best_route)):
                if j + 1 < len(best_route):
                    #Calculating the change in distance by swapping edges (i, i+1) and (j, j+1)
                    #By comparing the current edge distance with the possible swap
                    delta_distance = (
                        distance_matrix[best_route[i - 1]][best_route[j]]
                        + distance_matrix[best_route[i]][best_route[j + 1]]
                        - distance_matrix[best_route[i - 1]][best_route[i]]
                        - distance_matrix[best_route[j]][best_route[j + 1]]
                    )
                    if delta_distance < 0:
                        #The difference in the distance is negative hence swap the edges as the total distance is decreasing
                        best_route[i:j + 1] = reversed(best_route[i:j + 1])
                        improved = True
    return best_route

def two_opt(routes, distance_matrix):
    final_routes = []
    for route in routes:
        final_route = two_opt_algorithm(route, distance_matrix)
        final_routes.append(final_route)
    return final_routes

def calculate_route_distance(route, distance_matrix):
    total_distance = 0
    for i in range(len(route) - 1):
        total_distance += distance_matrix[route[i]][route[i + 1]]
    return total_distance

def calculate_total_distance(routes, distance_matrix):
    return sum(calculate_route_distance(route, distance_matrix) for route in routes)

if __name__ == "__main__":
    data = create_data_model()

    #Initial solution is set equal to the nearest neighbor algorithm solution
    initial_routes, initial_distances, total_distance_initial = nearest_neighbor_algorithm(data)

    #The results for the initial solution using Nearest Neighbor
    print("Nearest Neighbor:")
    for i, (route, distance) in enumerate(zip(initial_routes, initial_distances)):
        print(f"Route {i + 1}: {route}, Distance: {distance}")

    print(f"\nTotal Distance of Nearest Neighbor Solution: {total_distance_initial}\n")

    #Applying Two-Opt Algorithm
    final_routes = two_opt(initial_routes, data["distance_matrix"])
    final_distances = [calculate_route_distance(route, data["distance_matrix"]) for route in final_routes]
    final_distance_improved = sum(final_distances)

    #The results for the improved solution using Two-Opt
    print("Two-Opt:")
    for i, (route, distance) in enumerate(zip(final_routes, final_distances)):
        print(f"Route {i + 1}: {route}, Distance: {distance}")

    print(f"\nTotal Distance of Two-opt Solution: {final_distance_improved}")

Nearest Neighbor:
Route 1: [0, 7, 5, 6, 2, 0], Distance: 1780
Route 2: [0, 9, 8, 14, 10, 0], Distance: 1712
Route 3: [0, 13, 12, 11, 15, 0], Distance: 1712
Route 4: [0, 1, 4, 3, 16, 0], Distance: 2876

Total Distance of Nearest Neighbor Solution: 8080

Two-Opt:
Route 1: [0, 7, 5, 6, 2, 0], Distance: 1780
Route 2: [0, 9, 10, 14, 8, 0], Distance: 1552
Route 3: [0, 13, 15, 11, 12, 0], Distance: 1552
Route 4: [0, 1, 4, 3, 16, 0], Distance: 2876

Total Distance of Two-opt Solution: 7760
